In [1]:
!cat .env

STAGE=PRODUCTION


## Escolher a unidade

In [1]:
company = Company.objects.get(name__icontains="Teste Fluxo Imt")
company

<Company: f501d276-df2a-4296-b739-40e0b67adf68: Teste fluxo IMT>

In [2]:
ApprovalFlow.objects.get(company=company, target_model="reportings.Reporting")

<ApprovalFlow: [Teste fluxo IMT]: Fluxo Apontamentos - reportings.Reporting>

In [3]:
for permission in UserPermission.objects.filter(companies=company):    
    
    try:
        if 'Reporting' in permission.permissions.keys():
            permission.permissions['Reporting']['can_approve'] = True
        
        else:
            permission.permissions['reporting']['can_approve'] = True

        permission.save()
        print(permission)
    except Exception as e:
        pass

['Teste fluxo IMT'] - Teste fluxo IMT


In [4]:
flow = ApprovalFlow(
    name="Fluxo Apontamentos",
    target_model="reportings.Reporting",
    company=company
)
flow.save()

In [5]:
elaboracao = ApprovalStep(
    name="Criado",
    approval_flow=flow,
)
elaboracao.responsible_created_by = True
elaboracao.save()
# elaboracao.responsible_users.add(User.objects.get(username='rlcs'))
# for a in approvers:
#     elaboracao.responsible_users.add(a)

In [6]:
homolog_normal = ApprovalStep(
    name="Aguardando Aprovação",
    approval_flow=flow,
)
homolog_normal.save()
# homolog_normal.responsible_users.add(User.objects.get(username='rlcs'))

In [7]:
revisao = ApprovalStep(
    name="Em Revisão",
    approval_flow=flow,
)
revisao.responsible_created_by = True
revisao.save()
revisao.responsible_users.add(User.objects.get(username='rlcs'))

In [8]:
deferido = ApprovalStep(
    name="Aprovado",
    approval_flow=flow,
)
deferido.save()

In [9]:
indeferido = ApprovalStep(
    name="Cancelado",
    approval_flow=flow,
)
indeferido.save()

In [10]:
trans_homolog_normal = ApprovalTransition(
    name="Solicitar Análise",
    origin=elaboracao,
    destination=homolog_normal,
    condition={
        "===": [{
            "var": "request.action"
            }, "Solicitar Análise"]
    },
    button={"icon": "assignment", "color": "primary", "variant": "raised"},
    callback={"change_fields": [{"name": "editable", "value": False}]}
)
trans_homolog_normal.save()

In [11]:
trans_revisar_normal = ApprovalTransition(
    name="Solicitar Revisão",
    origin=homolog_normal,
    destination=revisao,
    condition={
        "===": [{
            "var": "request.action"
        }, "Solicitar Revisão"]
    },
    button={"icon": "warning", "color": "amber", "variant": "outlined", "needs_dialog": True},
    callback={"change_fields": [{"name": "editable", "value": True}]}
)
trans_revisar_normal.save()

In [12]:
trans_homolog_again_normal = ApprovalTransition(
    name="Solicitar Análise",
    origin=revisao,
    destination=homolog_normal,
    condition={
        "===": [{
            "var": "request.action"
        }, "Solicitar Análise"]
    },
    button={"icon": "assignment", "color": "primary", "variant": "raised"},
    callback={"change_fields": [{"name": "editable", "value": False}]}
)
trans_homolog_again_normal.save()

In [13]:
trans_homologar_normal = ApprovalTransition(
    name="Aprovar",
    origin=homolog_normal,
    destination=deferido,
    condition={
        "===": [{
            "var": "request.action"
        }, "Aprovar"] 
    },
    button={"icon": "check", "color": "primary", "variant": "raised"},
    callback={"change_fields": [{"name": "editable", "value": False}]}
)
trans_homologar_normal.save()

In [14]:
trans_rejeitar_normal = ApprovalTransition(
    name="Cancelar",
    origin=homolog_normal,
    destination=indeferido,
    condition={
        "===": [{
            "var": "request.action"
        }, "Cancelar"]
    },
    button={"icon": "clear", "color": "red", "variant": "outlined", "needs_dialog": True},
    callback={"change_fields": [{"name": "editable", "value": False}]}
)
trans_rejeitar_normal.save()

In [15]:
Reporting.objects.filter(company=company).update(approval_step=elaboracao)

0

In [16]:
str(deferido.uuid)

'ef9db6ec-5c7b-4904-abfb-209f6f7a1cc2'

In [17]:
company.metadata['approved_approval_steps'] = [str(deferido.uuid)]

In [18]:
company.save()

# Passos para outros tipos especiais de approval

In [24]:
at2 = ApprovalTransition(name='Solicitar revisão após aprovado',
                        origin=deferido, # apontamento Aprovado
                         destination=revisao, # apontamento Em Revisão
                         condition={'===': [{'var': 'request.action'}, 'Solicitar revisão após aprovado']},
                         callback={'change_fields': [{'name': 'editable', 'value': True}]},
                         button={'icon': 'check', 'color': 'primary', 'variant': 'raised', "needs_dialog": True}
                        )
at2.save()

In [33]:
homolog_normal.responsible_firms.add(*Firm.objects.filter(company=company,name='Aprovadores de Apontamentos'))

In [40]:
homolog_normal.responsible_users.all()

<QuerySet []>

In [19]:
aprovado_revisao = ApprovalStep(
    name="Aprovado/Revisão",
    approval_flow=ApprovalFlow.objects.get(company=company, target_model="reportings.Reporting"),
)
aprovado_revisao.save()

In [20]:
aprovado_revisao.responsible_users.add(User.objects.get(username='rlcs'))
aprovado_revisao.responsible_users.all()

<QuerySet [<User: rlcs - Kartado CS>]>

# Criação Approvals para OccurrenceRecord Energia

In [3]:
company = Company.objects.get(name__icontains="Demo Energia")
company

<Company: 5a7918ca-d799-4f6b-8b48-28bfab77a52a: Demo Energia>

In [4]:
ApprovalFlow.objects.get(company=company, target_model="occurrence_records.OccurrenceRecord")

DoesNotExist: ApprovalFlow matching query does not exist.

In [5]:
for permission in UserPermission.objects.filter(companies=company):    
    
    try:
        if 'OccurrenceRecord' in permission.permissions.keys():
            permission.permissions['OccurrenceRecord']['can_approve'] = True
        
        else:
            permission.permissions['OccurrenceRecord']['can_approve'] = True

        permission.save()
        print(permission)
    except Exception as e:
        pass

['Demo Energia'] - LÍDER TÉCNICO E COMERCIAL EXTERNO
['Demo Energia'] - ADMINISTRADOR
['Demo Energia'] - COLABORADOR INTERNO
['Demo Energia'] - HOMOLOGADOR


In [6]:
flow = ApprovalFlow(
    name="Fluxo Registros",
    target_model="occurrence_records.OccurrenceRecord",
    company=company
)
flow.save()

In [7]:
elaboracao = ApprovalStep(
    name="Registro em Elaboração",
    approval_flow=flow,
)
elaboracao.responsible_created_by = True
elaboracao.save()
elaboracao.responsible_users.add(User.objects.get(username='rlcs'))
# for a in approvers:
#     elaboracao.responsible_users.add(a)

In [8]:
aguard_aprov_tec = ApprovalStep(
    name="Aguardando aprovação técnica",
    approval_flow=flow,
)
aguard_aprov_tec.save()
aguard_aprov_tec.responsible_users.add(User.objects.get(username='rlcs'))

In [9]:
revisao = ApprovalStep(
    name="Necessita Revisão",
    approval_flow=flow,
)
revisao.responsible_created_by = True
revisao.save()
revisao.responsible_users.add(User.objects.get(username='rlcs'))

In [10]:
homologacao = ApprovalStep(
    name="Aguardando Homologação",
    approval_flow=flow,
)
homologacao.responsible_created_by = True
homologacao.save()
homologacao.responsible_users.add(User.objects.get(username='rlcs'))

In [11]:
atendido = ApprovalStep(
    name="Atendido",
    approval_flow=flow,
)
atendido.responsible_created_by = True
atendido.save()
atendido.responsible_users.add(User.objects.get(username='rlcs'))

In [12]:
reagendado = ApprovalStep(
    name="Reagendado",
    approval_flow=flow,
)
reagendado.responsible_created_by = True
reagendado.save()
reagendado.responsible_users.add(User.objects.get(username='rlcs'))

In [13]:
deferido = ApprovalStep(
    name="Registro Deferido",
    approval_flow=flow,
)
deferido.save()

In [14]:
cancelado = ApprovalStep(
    name="Cancelado",
    approval_flow=flow,
)
cancelado.save()

In [15]:
indeferido = ApprovalStep(
    name="Registro Indeferido",
    approval_flow=flow,
)
indeferido.save()

In [16]:
trans_homolog_normal = ApprovalTransition(
    name="Solicitar Homologação",
    origin=aguard_aprov_tec,
    destination=homologacao,
    condition={'and': [{'===': [{'var': 'request.action'}, 'Solicitar Homologação']}, {'!==': [{'var': 'source.relationships.occurrenceType.data.id'}, 'be716386-51aa-4026-ba80-b555dc29ddaa']}]},
    button={"icon": "assignment", "color": "primary", "variant": "raised"},
    callback={'change_fields': [{'name': 'status', 'value': {'type': 'object', 'model': 'service_orders.ServiceOrderActionStatus', 'filters': {'uuid': '20134139-c77b-4404-8b9c-6697af03d07c'}}}, {'name': 'responsible', 'value': {'type': 'object', 'model': 'companies.Firm', 'follow': {'key': 'manager', 'type': 'relationship'}, 'filters': {'uuid': 'ed9132c6-b426-44a4-973d-31d85081e40d'}}}]}
)
trans_homolog_normal.save()

In [17]:
trans_revisar_normal = ApprovalTransition(
    name="Solicitar revisão",
    origin=aguard_aprov_tec,
    destination=revisao,
    condition={
        "===": [{
            "var": "request.action"
        }, "Solicitar revisão"]
    },
    button={"icon": "warning", "color": "amber", "variant": "outlined", "needs_dialog": True},
    callback={'change_fields': [{'name': 'editable', 'value': True}, {'name': 'status', 'value': {'type': 'object', 'model': 'service_orders.ServiceOrderActionStatus', 'filters': {'uuid': '08199255-6eaf-4415-838b-56357287152e'}}}]}
)
trans_revisar_normal.save()

In [18]:
trans_homolog_again_normal = ApprovalTransition(
    name="Rejeitar",
    origin=homologacao,
    destination=indeferido,
    condition={'and': [{'===': [{'var': 'request.action'}, 'Rejeitar']}, {'!==': [{'var': 'source.relationships.occurrenceType.data.id'}, 'be716386-51aa-4026-ba80-b555dc29ddaa']}]},
    button={'icon': 'cancel', 'color': 'red', 'variant': 'outlined'},
    callback={'change_fields': [{'name': 'status', 'value': {'type': 'object', 'model': 'service_orders.ServiceOrderActionStatus', 'filters': {'uuid': 'ff9face2-8bd7-4ab3-8f1d-8df52d733574'}}}]}
)
trans_homolog_again_normal.save()

In [19]:
trans_homologar_normal = ApprovalTransition(
    name="Solicitar Revisão",
    origin=homologacao,
    destination=revisao,
    condition={'and': [{'===': [{'var': 'request.action'}, 'Solicitar Revisão']}, {'!==': [{'var': 'source.relationships.occurrenceType.data.id'}, 'be716386-51aa-4026-ba80-b555dc29ddaa']}]},
    button={'icon': 'warning', 'color': 'amber', 'variant': 'outlined', 'needs_dialog': True},
    callback={'change_fields': [{'name': 'status', 'value': {'type': 'object', 'model': 'service_orders.ServiceOrderActionStatus', 'filters': {'uuid': '08199255-6eaf-4415-838b-56357287152e'}}}, {'name': 'responsible', 'value': {'key': 'created_by', 'type': 'relationship'}}]}
)
trans_homologar_normal.save()

In [20]:
trans_rejeitar_normal = ApprovalTransition(
    name="Rejeitar",
    origin=homologacao,
    destination=indeferido,
    condition={'and': [{'===': [{'var': 'request.action'}, 'Rejeitar']}, {'===': [{'var': 'source.relationships.occurrenceType.data.id'}, 'be716386-51aa-4026-ba80-b555dc29ddaa']}]},
    button={'icon': 'cancel', 'color': 'red', 'variant': 'outlined'},
    callback={'change_fields': [{'name': 'status', 'value': {'type': 'object', 'model': 'service_orders.ServiceOrderActionStatus', 'filters': {'uuid': 'ff9face2-8bd7-4ab3-8f1d-8df52d733574'}}}]}
)
trans_rejeitar_normal.save()

In [21]:
trans_deferir_normal = ApprovalTransition(
    name="Homologar",
    origin=homologacao,
    destination=deferido,
    condition={'and': [{'===': [{'var': 'request.action'}, 'Homologar']}, {'!==': [{'var': 'source.relationships.occurrenceType.data.id'}, 'be716386-51aa-4026-ba80-b555dc29ddaa']}]},
    button={'icon': 'check', 'color': 'primary', 'variant': 'raised', 'to_action': 'Sinalizar como Atendido', 'check_procedures': True},
    callback={'is_approved': True, 'change_fields': [{'name': 'status', 'value': {'type': 'object', 'model': 'service_orders.ServiceOrderActionStatus', 'filters': {'uuid': '4f89dd8b-dbe9-4cae-be66-ea7ffa2af61b'}}}], 'create_objects': True}
)
trans_rejeitar_normal.save()

In [22]:
str(deferido.uuid)

'69e3e477-4a07-4ae1-9d98-1425364daecd'

In [23]:
company.metadata['approved_approval_steps'] = [str(deferido.uuid)]

In [24]:
company.save()